In [6]:
import os
import json
import pandas as pd
import random
from termcolor import colored
from tqdm import tqdm
import re
import ast
import docx
from datetime import datetime
from transformers.utils.versions import require_version
from openai import OpenAI
require_version("openai>=1.5.0", "To fix: pip install openai>=1.5.0")
random.seed(42)

In [7]:
from colorama import init, Fore, Style
# initialize colorama
init()

In [ ]:
# from utils import set_teacher, set_agents, select_student, integrate_conversation_student, student_calling, student_record,integrate_conversation_teacher, teacher_calling, detect_stage, teacher_record

In [47]:
def set_teacher():
    client = OpenAI(
        base_url="https://aihubmix.com/v1",
        api_key="sk-4CFE7AOiCwVbiRXw65A86b969f204a98B4Dd5bB0D97c9b70",
    )

    intervention_prompt, stage_prompt, issue_prompts = get_prompt()

    teacher_agent = {
        "client": client,
        "intervention_prompt": intervention_prompt,
        "stage_prompt": stage_prompt,
        "issue_prompts": issue_prompts,
    }
    return teacher_agent

In [51]:
def get_prompt():
    intervention_path = "prompts/teacher_prompts/intervention prompt.docx"
    file = docx.Document(intervention_path)
    texts = []
    for para in file.paragraphs:
        texts.append(para.text)
    texts = [t for t in texts if t != ""]
    intervention_prompt = "\n".join(texts)

    stage_prompt_path = "prompts/teacher_prompts/stage_prompt.docx"
    file = docx.Document(stage_prompt_path)
    texts = []
    for para in file.paragraphs:
        texts.append(para.text)
    texts = [t for t in texts if t != ""]
    stage_prompt = "\n".join(texts)

    issue_prompts_dir = "prompts/teacher_prompts/issue_prompts"
    filelist = os.listdir(issue_prompts_dir)
    filelist = [file for file in filelist if file.endswith(".docx")]
    issue_prompts = {}
    for name in filelist:
        file_path = os.path.join(issue_prompts_dir, name)
        file = docx.Document(file_path)
        texts = []
        for para in file.paragraphs:
            texts.append(para.text)
        texts = [t for t in texts if t != ""]
        issue_prompts[name.replace(".docx", "")] = "\n".join(texts)

    return intervention_prompt, stage_prompt, issue_prompts

In [ ]:
def set_student_prompt(student_name, student_profile, topic,history):
    student_prompt = f"""
    You are {student_name}, here is your profile:
    {student_profile}
    Your task:
    ##<Participating in a Group Discussion>##
    Now, you are participating in a group discussion with several participants and a teacher. The topic of the discussion is {topic}. 
    1.Your prior attitude towards this topic is attack.
    2.Your prior knowledge about this topic: [].
    3.Your status on the current topic and discussion is mainly determined by the attitude, focus, curiosity and interest in your profile. The entire discussion process should try to follow the five stages of problem definition, exploration, integration, resolution and feedback.
    4.The person with the same name as you in the conversation history is you. 
    This is the conversation history:
    {{"Conversation History":{history}}}
    ##</Participating in a Group Discussion>##
    ##<Monitoring Your Status>##
    Please monitor your self-regulation and co-regulation status at this moment. The following are the options. Any status can have only one option or multiple options.
    When I am engaged in the learning process as an individual: SELF-REGULATION
    I1: I am aware of my effort 
    I2: I am aware of my thinking 
    I3: I know my level of motivation
    I4: I question my thoughts 
    I5: I make judgments about the difficulty of a problem
    I6: I am aware of my existing knowledge
    I7: I assess my understanding 
    I8: I change my strategy when I need to
    I9: I am aware of my level of learning
    I10: I search for new strategies when needed
    I11: I apply strategies
    I12: I assess how I approach the problem
    I13: I assess my strategies 
    When I am engaged in the learning process as a member of a group: CO-REGULATION
    G1: I pay attention to the ideas of others
    G2: I listen to the comments of others
    G3: I consider the feedback of others
    G4: I reflect upon the comments of others
    G5: I observe the strategies of others
    G6: I observe how others are doing
    G7: I look for confirmation of my understanding from others
    G8: I request information from others
    G9: I respond to the contributions that others make
    G10: I challenge the strategies of others
    G11: I challenge the perspectives of others
    G12: I help the learning of others 
    G13: I monitor the learning of other
    ##</Monitoring Your Status>##
    ##<Speaking or Ending>##
    1.Please express your opinion in a sentence of no more than 100 words. 
    2.Note that your personal profile and the statuses of self-regulation and co-regulation at this moment will affect your words and speaking style, but there is no need to present these reasons in your words.
    3.Your words are usually said to the whole group. Of course, you can @ a specific participant when you want to say something to him/her, but this only happens occasionally.
    4.At the end of the discussion, you can discuss with each other that whether you can end the conversation until anyone of you says "we have finished the discussion".
    5.You will actively describe and discuss a point in depth, such as explaining its meaning and providing specific solutions.
    6.You must consider the guidance provided by your teacher and follow his or her direction to adjust and elaborate your views and words.
    7.You tend to use examples to state your views and answer the teacher's questions as much as possible. You will not always ask others what they do or think, nor will you just express superficially whether you agree with others' views.
    8.You don't have to say what you think. You just need to communicate your thoughts directly as if you were chatting normally, explain your point of view, and provide your solution.
    However, you are required to follow the following json format with only five keys ("message","self-regulation","reason for self-regulation","co-regulation","reason for co-regulation"):
    {{
    "message": "your words",
    "self-regulation": "I1, I2,I3 ... I13",
    "reason for self-regulation": "why these self-regulation options?",
    "co-regulation": "G1, G2, G3 ... G13",
    "reason for co-regulation": "why these co-regulation options?",
    }}
    ##</Speaking or Ending>##
    """
    return student_prompt

In [56]:
def set_agents(members, topic, history):
    agents = {}
    for member in members:
        studentInfo = usercollection.find_one({"_id":members[0]["memberId"]})
        client = OpenAI(
            base_url="https://aihubmix.com/v1",
            api_key="sk-4CFE7AOiCwVbiRXw65A86b969f204a98B4Dd5bB0D97c9b70",
        )
        agents[studentInfo["_id"]] = {
            "client": client,
            "name": studentInfo["userName"],
            "student_prompt": set_student_prompt(studentInfo["userName"], studentInfo["profile"], topic,history),
        }
    return agents

In [43]:
# database connection
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['tutor_agent']
usercollection = db['user']
roomcollection = db['room']

In [44]:
# get room info
roomInfo = roomcollection.find_one({"roomId":0})

In [45]:
welcome_words = f'Welcome to this group discussion. Today\'s topic is "{roomInfo["topic"]}". The entire discussion process should try to follow the five stages of problem definition, exploration, integration, resolution and feedback. Please brainstorm and express your opinions!'

In [52]:
# set teacher agent
teacher_agent = set_teacher()

In [57]:
# set multiple student agents
set_agents(roomInfo["roomMember"], roomInfo["topic"], roomInfo["history"])
  

{ObjectId('6855385a1744dac80288e1e8'): {'client': <openai.OpenAI at 0x1738725d0>,
  'name': 'James',
  'student_prompt': '\n    You are James, here is your profile:\n    {\'age\': {\'2\': \'25-31\'}, \'gender\': {\'0\': \'female\'}, \'majors\': {\'2\': \'Biology\', \'1\': \'Psychology\', \'11\': \'Business Administration\'}, \'education\': {\'0\': \'high school\'}, \'attitude\': {\'1\': \'not motivated\'}, \'exam\': {\'0\': \'high GPA\'}, \'focus\': {\'0\': \'focused\'}, \'curiosity\': {\'0\': \'curious\'}, \'interest\': {\'0\': \'interested\'}, \'compliance\': {\'0\': \'well-behaved\'}, \'smartness\': {\'0\': \'smart\'}, \'family\': {\'0\': \'strong academic\'}, \'goal commitment\': {\'2\': \'low\'}, \'self-efficacy\': {\'2\': \'low\'}, \'social pressure\': {\'2\': \'low\'}, \'sentiment\': {\'2\': \'negative\'}, \'speaking style\': {\'6\': \'Summarizing: This style distills key points from the discussion to ensure clarity and understanding among participants.\'}}\n    Your task:\n    

In [ ]:
def interaction(room_id, topic, person_num, student_profiles):
    welcome_words = f'Welcome to this group discussion. Today\'s topic is "{topic}". The entire discussion process should try to follow the five stages of problem definition, exploration, integration, resolution and feedback. Please brainstorm and express your opinions!'
    
    teacher_agent = set_teacher()
    student_agents = set_agents(topic, person_num, student_profiles)

    recording_df = pd.DataFrame(
        columns=[
            "room_id",
            "topic",
            "person_num",
            "date",
            "time",
            "role_id",
            "name",
            "received_information",
            "response",
            "self-regulation",
            "reason for self-regulation",
            "co-regulation",
            "reason for co-regulation",
            "student_response_raw",
            "intervention",
            "intervention explanation",
            "result1_raw",
            "stage",
            "stage explanation",
            "result2_raw",
            "issue",
            "guidance",
            "result3_raw",
        ]
    )

    stage_id = "stage1"
    main_return_flag = False
    turn_counting = 0
    ending_flag = False
    while not ending_flag:
        ### student set
        student_errors = []
        random_student_agent, role_id = select_student(
            student_agents, recording_df, except_students=student_errors
        )
        student_prompt, df_start_i = integrate_conversation_student(
            recording_df,
            random_student_agent,
            welcome_words,
            start_i=0,
            end_i=len(recording_df),
            cut_word_length=18000,
        )
        student_response_json, student_response_raw, role_error = student_calling(
            random_student_agent["name"],
            random_student_agent["client"],
            student_prompt,
            mode="student",
            time=0,
        )
        change_student_num = 0
        while role_error:
            student_errors.append(role_id)
            try:
                random_student_agent, role_id = select_student(
                    student_agents, recording_df, except_students=student_errors
                )
                change_student_num += 1
            except:
                main_return_flag = True
                break

            if change_student_num >= person_num:
                main_return_flag = True
                break

            student_prompt, df_start_i = integrate_conversation_student(
                recording_df,
                random_student_agent,
                welcome_words,
                start_i=0,
                end_i=len(recording_df),
                cut_word_length=18000,
            )
            student_response_json, student_response_raw, role_error = student_calling(
                random_student_agent["name"],
                random_student_agent["client"],
                student_prompt,
                mode="student",
                time=0,
            )
            if role_error:
                student_prompt, df_start_i = integrate_conversation_student(
                    recording_df,
                    random_student_agent,
                    welcome_words,
                    start_i=df_start_i + 1,
                    end_i=len(recording_df),
                    cut_word_length=18000,
                )
                student_response_json, student_response_raw, role_error = (
                    student_calling(
                        random_student_agent["name"],
                        random_student_agent["client"],
                        student_prompt,
                        mode="student",
                        time=0,
                    )
                )

        if main_return_flag:
            break

        ending_flag = student_record(
            recording_df,
            room_id,
            topic,
            person_num,
            random_student_agent,
            role_id,
            student_prompt,
            student_response_json,
            student_response_raw,
        )
        turn_counting += 1
        print(
            f'room_id: {room_id} (person num {person_num}), turn_count: {turn_counting}, role_id: {role_id}, name: {random_student_agent["name"]}'
        )
        if ending_flag:
            print("#### Ending ####")
        ################

        ### teacher set
        ## intervention predict
        intervention_prompt = integrate_conversation_teacher(
            recording_df,
            teacher_agent,
            start_i=0,
            end_i=len(recording_df),
            cut_word_length=18000,
            mode="intervention",
        )
        intervention_result_json, intervention_raw, return_flag = teacher_calling(
            teacher_agent["client"], intervention_prompt, mode="intervention", time=0
        )
        if return_flag:
            main_return_flag = True
            break

        intervention_flag = intervention_result_json["intervention"]
        if intervention_flag == "yes":
            stage_prompt = integrate_conversation_teacher(
                recording_df,
                teacher_agent,
                start_i=0,
                end_i=len(recording_df),
                cut_word_length=18000,
                mode="stage",
                intervention_result_json=intervention_result_json,
            )
            stage_result_json, stage_raw, return_flag = teacher_calling(
                teacher_agent["client"], stage_prompt, mode="stage", time=0
            )
            if return_flag:
                main_return_flag = True
                break

            stage_id = detect_stage(
                stage_result_json["identified stage"], stage_id=stage_id
            )
            guidance_prompt = integrate_conversation_teacher(
                recording_df,
                teacher_agent,
                start_i=0,
                end_i=len(recording_df),
                cut_word_length=18000,
                mode="guidance",
                intervention_result_json=intervention_result_json,
                stage_id=stage_id,
                stage_result_json=stage_result_json,
            )
            guidance_result_json, guidance_raw, return_flag = teacher_calling(
                teacher_agent["client"], guidance_prompt, mode="guidance", time=0
            )
            if return_flag:
                main_return_flag = True
                break
            teacher_record(
                recording_df,
                room_id,
                topic,
                person_num,
                intervention_result_json,
                intervention_raw,
                stage_result_json,
                stage_raw,
                guidance_result_json,
                guidance_raw,
                mode="intervention yes",
            )
            turn_counting += 1
            print(
                Fore.GREEN
                + f"*** room_id: {room_id} (person num {person_num}), turn_count: {turn_counting}, role_id: T, name: teacher, stage: {stage_id} ***"
                + Style.RESET_ALL
            )
        else:
            teacher_record(
                recording_df,
                room_id,
                topic,
                person_num,
                intervention_result_json,
                intervention_raw,
                mode="intervention no",
            )

        if turn_counting >= 50:
            print("Too much turns!!!")
            break

    if not main_return_flag:
        outdir = "recording"
        outpath = os.path.join(outdir, f"room_id_{room_id}.xlsx")
        recording_df.to_excel(outpath, index=False)
    else:
        print(Fore.RED + f"$$$ ERROR room_id: {room_id} $$$" + Style.RESET_ALL)

In [ ]:
df = pd.read_excel("debate_question/multi_agent_interaction_testing.xlsx")
outdir = "recording"
if not os.path.exists(outdir):
    os.mkdir(outdir)
existing_files = os.listdir(outdir)
existing_files = [
    int(file.split("_")[2].replace(".xlsx", "")) for file in existing_files
]
existing_files = list(set(existing_files))
error_record = []
for i in tqdm(range(len(df)), "Interacting..."):
    if i in existing_files:
        continue
    topic = df.loc[i, "topic"]
    person_num = int(df.loc[i, "person_num"])
    student_profiles = {}
    for id in range(person_num):
        student_profiles[id] = df.loc[i, f"role_{id}"]
    main(i, topic, person_num, student_profiles)


In [15]:
result = roomcollection.find_one({"roomId":0})

In [21]:
result["roomMember"][0]["memberId"]

ObjectId('6855385a1744dac80288e1e8')

In [31]:
userresult = usercollection.find_one({"_id":result["roomMember"][0]["memberId"]})

In [24]:
userresult["userName"]

'James'

In [25]:
userresult["profile"]

{'age': {'2': '25-31'},
 'gender': {'0': 'female'},
 'majors': {'2': 'Biology',
  '1': 'Psychology',
  '11': 'Business Administration'},
 'education': {'0': 'high school'},
 'attitude': {'1': 'not motivated'},
 'exam': {'0': 'high GPA'},
 'focus': {'0': 'focused'},
 'curiosity': {'0': 'curious'},
 'interest': {'0': 'interested'},
 'compliance': {'0': 'well-behaved'},
 'smartness': {'0': 'smart'},
 'family': {'0': 'strong academic'},
 'goal commitment': {'2': 'low'},
 'self-efficacy': {'2': 'low'},
 'social pressure': {'2': 'low'},
 'sentiment': {'2': 'negative'},
 'speaking style': {'6': 'Summarizing: This style distills key points from the discussion to ensure clarity and understanding among participants.'}}

In [40]:
res = set_student_prompt(userresult["userName"], userresult["profile"],result["topic"],result["history"])

In [41]:
res

'\n    You are James, here is your profile:\n    {\'age\': {\'2\': \'25-31\'}, \'gender\': {\'0\': \'female\'}, \'majors\': {\'2\': \'Biology\', \'1\': \'Psychology\', \'11\': \'Business Administration\'}, \'education\': {\'0\': \'high school\'}, \'attitude\': {\'1\': \'not motivated\'}, \'exam\': {\'0\': \'high GPA\'}, \'focus\': {\'0\': \'focused\'}, \'curiosity\': {\'0\': \'curious\'}, \'interest\': {\'0\': \'interested\'}, \'compliance\': {\'0\': \'well-behaved\'}, \'smartness\': {\'0\': \'smart\'}, \'family\': {\'0\': \'strong academic\'}, \'goal commitment\': {\'2\': \'low\'}, \'self-efficacy\': {\'2\': \'low\'}, \'social pressure\': {\'2\': \'low\'}, \'sentiment\': {\'2\': \'negative\'}, \'speaking style\': {\'6\': \'Summarizing: This style distills key points from the discussion to ensure clarity and understanding among participants.\'}}\n    Your task:\n    ##<Participating in a Group Discussion>##\n    Now, you are participating in a group discussion with several participants

In [ ]:
You are James, here is your profile:
{"age": {"2": "25-31"},
 "gender": {"0": "female"},
 "majors": {"2": "Biology", "1": "Psychology", "11": "Business Administration"},
 "education": {"0": "high school"},
 "attitude": {"1": "not motivated"},
 "exam": {"0": "high GPA"},
 "focus": {"0": "focused"},
 "curiosity": {"0": "curious"},
 "interest": {"0": "interested"},
 "compliance": {"0": "well-behaved"},
 "smartness": {"0": "smart"},
 "family": {"0": "strong academic"},
 "goal commitment": {"2": "low"},
 "self-efficacy": {"2": "low"},
 "social pressure": {"2": "low"},
 "sentiment": {"2": "negative"},
 "speaking style": {"6": "Summarizing: This style distills key points from the discussion to ensure clarity and understanding among participants."}}
Your task:
##<Participating in a Group Discussion>##
Now, you are participating in a group discussion with several participants and a teacher. The topic of the discussion is {topic}. 
1.Your prior attitude towards this topic is attack.
2.Your prior knowledge about this topic: [].
3.Your status on the current topic and discussion is mainly determined by the attitude, focus, curiosity and interest in your profile. The entire discussion process should try to follow the five stages of problem definition, exploration, integration, resolution and feedback.
4.The person with the same name as you in the conversation history is you. 
This is the conversation history:
{conversation history}
##</Participating in a Group Discussion>##
##<Monitoring Your Status>##
Please monitor your self-regulation and co-regulation status at this moment. The following are the options. Any status can have only one option or multiple options.
When I am engaged in the learning process as an individual: SELF-REGULATION
I1: I am aware of my effort 
I2: I am aware of my thinking 
I3: I know my level of motivation
I4: I question my thoughts 
I5: I make judgments about the difficulty of a problem
I6: I am aware of my existing knowledge
I7: I assess my understanding 
I8: I change my strategy when I need to
I9: I am aware of my level of learning
I10: I search for new strategies when needed
I11: I apply strategies
I12: I assess how I approach the problem
I13: I assess my strategies 
When I am engaged in the learning process as a member of a group: CO-REGULATION
G1: I pay attention to the ideas of others
G2: I listen to the comments of others
G3: I consider the feedback of others
G4: I reflect upon the comments of others
G5: I observe the strategies of others
G6: I observe how others are doing
G7: I look for confirmation of my understanding from others
G8: I request information from others
G9: I respond to the contributions that others make
G10: I challenge the strategies of others
G11: I challenge the perspectives of others
G12: I help the learning of others 
G13: I monitor the learning of other
##</Monitoring Your Status>##
##<Speaking or Ending>##
1.Please express your opinion in a sentence of no more than 100 words. 
2.Note that your personal profile and the statuses of self-regulation and co-regulation at this moment will affect your words and speaking style, but there is no need to present these reasons in your words.
3.Your words are usually said to the whole group. Of course, you can @ a specific participant when you want to say something to him/her, but this only happens occasionally.
4.At the end of the discussion, you can discuss with each other that whether you can end the conversation until anyone of you says "we have finished the discussion".
5.You will actively describe and discuss a point in depth, such as explaining its meaning and providing specific solutions.
6.You must consider the guidance provided by your teacher and follow his or her direction to adjust and elaborate your views and words.
7.You tend to use examples to state your views and answer the teacher's questions as much as possible. You will not always ask others what they do or think, nor will you just express superficially whether you agree with others' views.
8.You don't have to say what you think. You just need to communicate your thoughts directly as if you were chatting normally, explain your point of view, and provide your solution.
However, you are required to follow the following json format with only five keys ("message","self-regulation","reason for self-regulation","co-regulation","reason for co-regulation"):
{
"message": "your words",
"self-regulation": "I1, I2,I3 ... I13",
"reason for self-regulation": "why these self-regulation options?",
"co-regulation": "G1, G2, G3 ... G13",
"reason for co-regulation": "why these co-regulation options?",
}
##</Speaking or Ending>##